In [5]:
## testscript
from nba_api.stats.endpoints import leaguedashplayerstats
import pandas as pd

stats = leaguedashplayerstats.LeagueDashPlayerStats(season="2023-24")
df = stats.get_data_frames()[0]
df.head()


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
0,1630639,A.J. Lawson,A.J.,1610612742,DAL,23.0,42,27,15,0.643,...,211,148,415,385,192,400,257,38,405,1
1,1631260,AJ Green,AJ,1610612749,MIL,24.0,56,35,21,0.625,...,121,223,375,313,145,369,257,38,343,1
2,1631100,AJ Griffin,AJ,1610612737,ATL,20.0,20,8,12,0.400,...,121,77,519,450,396,480,257,38,469,1
3,203932,Aaron Gordon,Aaron,1610612743,DEN,28.0,73,49,24,0.671,...,520,457,52,93,13,79,54,38,87,1
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,27.0,78,39,39,0.500,...,335,419,238,216,123,243,257,38,233,1


In [ ]:
## Get the top 20 PTS players
top_scorers = df.sort_values("PTS", ascending=False).head(20)
top_scorers[["PLAYER_NAME","TEAM_ABBREVIATION","GP","PTS","REB","AST"]]


,PLAYER_NAME,TEAM_ABBREVIATION,GP,PTS,REB,AST
372,Luka Dončić,DAL,70,2370,647,686
501,Shai Gilgeous-Alexander,OKC,75,2254,415,465
184,Giannis Antetokounmpo,MIL,73,2222,841,476
233,Jalen Brunson,NYK,77,2212,278,519
434,Nikola Jokić,DEN,79,2085,976,708
29,Anthony Edwards,MIN,79,2049,430,405
337,Kevin Durant,PHX,75,2032,495,378
268,Jayson Tatum,BOS,74,1987,601,364
125,De'Aaron Fox,SAC,74,1966,340,418
509,Stephen Curry,GSW,74,1956,330,379
